In [1]:
import pickle
import torch
from torch_geometric.data import Data,DataLoader
from functions_refactor import *
from pytorch_util import *
from torch.optim import Adam

In [3]:
# model parameters
reuse = False
block = MEGNet_block
head = feedforwardHead_Update
data = '../Data/{}_data_ACSF_expand_PCA.pickle'
batch_size = 32
dim = 128
epochs = 20
clip = 0.4
layer1 = 3
layer2 = 3
factor = 2
lr = 1e-4

#### train base

In [4]:
train_dl,val_dl = get_data(data,batch_size)

model = GNN_edgeUpdate(reuse,block,head,dim,layer1,layer2,factor,**data_dict[data]).to('cuda:0')

paras = trainable_parameter(model)
opt = Adam(paras,lr=lr)

model,train_loss_list,val_loss_list,bestWeight = train_type(opt,model,epochs,train_dl,val_dl,paras,clip)

epoch:0, train_loss: +0.411, val_loss: -0.013, 
train_vector: +4.31|+1.21|+0.15|-0.37|-0.59|+0.12|-0.59|-0.95, 
val_vector  : +4.06|+0.39|-0.25|-0.88|-0.98|-0.18|-0.99|-1.29

epoch:1, train_loss: -0.201, val_loss: -0.523, 
train_vector: +3.08|+0.34|-0.32|-0.95|-1.12|-0.24|-1.04|-1.37, 
val_vector  : +0.98|+0.24|-0.48|-0.74|-1.22|-0.37|-1.20|-1.41

epoch:2, train_loss: -0.629, val_loss: -0.729, 
train_vector: +0.92|+0.13|-0.48|-1.19|-1.31|-0.38|-1.18|-1.53, 
val_vector  : +0.45|+0.46|-0.64|-1.36|-1.37|-0.46|-1.34|-1.56

epoch:3, train_loss: -0.750, val_loss: -0.849, 
train_vector: +0.81|-0.06|-0.60|-1.31|-1.44|-0.47|-1.29|-1.65, 
val_vector  : +0.71|-0.23|-0.67|-1.52|-1.42|-0.54|-1.41|-1.72

epoch:4, train_loss: -0.826, val_loss: -0.980, 
train_vector: +0.77|-0.13|-0.68|-1.42|-1.52|-0.53|-1.36|-1.74, 
val_vector  : +0.29|-0.51|-0.82|-1.32|-1.64|-0.61|-1.46|-1.78

epoch:5, train_loss: -0.893, val_loss: -1.079, 
train_vector: +0.73|-0.23|-0.74|-1.50|-1.60|-0.58|-1.43|-1.80, 
val_vector  :

RuntimeError: Error(s) in loading state_dict for GNN_edgeUpdate:
	Missing key(s) in state_dict: "head.linear.0.weight", "head.linear.0.bias", "head.linear.2.weight", "head.linear.2.bias". 
	Unexpected key(s) in state_dict: "head.linear.4.weight", "head.linear.4.bias", "head.linear.1.weight", "head.linear.1.bias". 

In [5]:
save_results(train_loss_perType,val_loss_perType,reuse,block,\
             head,data,batch_size,dim,clip,layer1,layer2,factor,epochs)
save_model_type(bestWeight,opt,reuse,block,head,data,batch_size,dim,clip,layer1,layer2,factor,epochs)

In [4]:
# this is using old definition of GNN_edgeUpdate shown below
train_dl,val_dl = get_data(data,batch_size)

model = GNN_edgeUpdate(reuse,block,dim,layer1,layer2,factor,**data_dict[data]).to('cuda:0')

paras = trainable_parameter(model)
opt = Adam(paras,lr=lr)

model,train_loss_perType,val_loss_perType = train(opt,model,epochs,train_dl,val_dl,paras,clip)

epoch:0, train_loss: +0.342, val_loss: -0.065, 
train_vector: +4.23|+0.92|+0.10|-0.45|-0.58|+0.11|-0.64|-0.96, 
val_vector  : +3.92|+0.12|-0.29|-0.89|-0.97|-0.18|-0.99|-1.24

epoch:1, train_loss: -0.261, val_loss: -0.644, 
train_vector: +2.65|+0.30|-0.34|-1.01|-1.09|-0.23|-1.06|-1.32, 
val_vector  : +0.58|-0.09|-0.52|-1.00|-1.14|-0.36|-1.21|-1.41

epoch:2, train_loss: -0.631, val_loss: -0.785, 
train_vector: +0.88|+0.14|-0.51|-1.24|-1.28|-0.36|-1.20|-1.48, 
val_vector  : +0.61|-0.26|-0.65|-1.47|-1.35|-0.43|-1.18|-1.55

epoch:3, train_loss: -0.747, val_loss: -0.930, 
train_vector: +0.80|-0.04|-0.62|-1.36|-1.40|-0.45|-1.31|-1.59, 
val_vector  : +0.23|-0.13|-0.75|-1.61|-1.47|-0.54|-1.47|-1.69

epoch:4, train_loss: -0.832, val_loss: -0.942, 
train_vector: +0.75|-0.17|-0.69|-1.47|-1.50|-0.52|-1.39|-1.67, 
val_vector  : +0.29|-0.09|-0.75|-1.62|-1.55|-0.59|-1.48|-1.74

epoch:5, train_loss: -0.887, val_loss: -1.066, 
train_vector: +0.73|-0.20|-0.76|-1.54|-1.56|-0.57|-1.45|-1.75, 
val_vector  :

In [5]:
save_results(train_loss_perType,val_loss_perType,reuse,block,\
             head,data,batch_size,dim,clip,layer1,layer2,factor,epochs)
save_model(model,opt,reuse,block,head,data,batch_size,dim,clip,layer1,layer2,factor,epochs)

Continue training

In [10]:
train_dl,val_dl = get_data(data,batch_size)

In [12]:
class GNN_edgeUpdate(torch.nn.Module):

    def __init__(self,reuse,block,dim,layer1,layer2,factor,\
                 node_in,edge_in,edge_in4,edge_in3=8):
        # block,head are nn.Module
        # node_in,edge_in are dim for bonding and edge_in4,edge_in3 for coupling
        super(GNN_edgeUpdate, self).__init__()
        self.lin_node = Sequential(BatchNorm1d(node_in),Linear(node_in, dim*factor),LeakyReLU(), \
                                   BatchNorm1d(dim*factor),Linear(dim*factor, dim),LeakyReLU())

        self.edge1 = Sequential(BatchNorm1d(edge_in),Linear(edge_in, dim*factor),LeakyReLU(), \
                                   BatchNorm1d(dim*factor),Linear(dim*factor, dim),LeakyReLU())

        self.edge2 = Sequential(BatchNorm1d(edge_in4+edge_in3),Linear(edge_in4+edge_in3, dim*factor),LeakyReLU(), \
                                   BatchNorm1d(dim*factor),Linear(dim*factor, dim),LeakyReLU())        
        if reuse:
            self.conv1 = block(dim=dim,edge_dim=edge_in)
            self.conv2 = block(dim=dim,edge_dim=edge_in3+edge_in4)
        else:
            self.conv1 = nn.ModuleList([block(dim=dim) for _ in range(layer1)])
            self.conv2 = nn.ModuleList([block(dim=dim) for _ in range(layer2)])            
        
        self.head = Sequential(Linear(dim, dim*factor),ReLU(), \
                               Linear(dim*factor, 1))
        
    def forward(self, data,IsTrain=False,typeTrain=False,weight=None):
        out = self.lin_node(data.x)
        # edge_*3 only does not repeat for undirected graph. Hence need to add (j,i) to (i,j) in edges
        edge_index3 = torch.cat([data.edge_index3,data.edge_index3[[1,0]]],1)
        n = data.edge_attr3.shape[0]
        temp_ = self.edge2(torch.cat([data.edge_attr3,data.edge_attr4],1))
        edge_attr3 = torch.cat([temp_,temp_],0)
        
        edge_attr = self.edge1(data.edge_attr)
        for conv in self.conv1:
            out,edge_attr = conv(out,data.edge_index,edge_attr)
        
        for conv in self.conv2:
            out,edge_attr3 = conv(out,edge_index3,edge_attr3)    
        
        edge_attr3 = edge_attr3[:n]
        if typeTrain:
            if IsTrain:
                y = data.y[data.type_attr]
            edge_attr3 = edge_attr3[data.type_attr]
            edge_attr3_old = data.edge_attr3[data.type_attr]
        else:
            if IsTrain:
                y = data.y
            edge_attr3_old = data.edge_attr3
            
        yhat = self.head(edge_attr3).squeeze(1)
        
        if IsTrain:
            k = torch.sum(edge_attr3_old,0)
            nonzeroIndex = torch.nonzero(k).squeeze(1)
            abs_ = torch.abs(y-yhat).unsqueeze(1)
            loss_perType = torch.zeros(8,device='cuda:0')
            loss_perType[nonzeroIndex] = torch.log(torch.sum(abs_ * edge_attr3_old[:,nonzeroIndex],0)/k[nonzeroIndex])
            loss = torch.sum(loss_perType)/nonzeroIndex.shape[0] if weight is None else torch.sum(loss_perType*weight)/nonzeroIndex.shape[0]
            return loss,loss_perType
        else:
            return yhat
        
def train(opt,model,epochs,train_dl,val_dl,paras,clip,typeTrain=False,train_loss_list=None,val_loss_list=None,weight=None):
    since = time.time()
    
    lossBest = 1e6
    if train_loss_list is None:
        train_loss_list,val_loss_list = [],[]
        epoch0 = 0
    else:
        epoch0 = len(train_loss_list)
        
    opt.zero_grad()
    for epoch in range(epochs):
        # training #
        model.train()
        np.random.seed()
        train_loss = 0
        train_loss_perType = np.zeros(8)
        val_loss = 0
        val_loss_perType = np.zeros(8)
        
        for i,data in enumerate(train_dl):
            data = data.to('cuda:0')
            loss,loss_perType = model(data,True,typeTrain,weight)
            loss.backward()
            clip_grad_value_(paras,clip)
            opt.step()
            opt.zero_grad()
            train_loss += loss.item()
            train_loss_perType += loss_perType.cpu().detach().numpy()
            
        # evaluating #
        model.eval()
        with torch.no_grad():
            for j,data in enumerate(val_dl):
                data = data.to('cuda:0')
                loss,loss_perType = model(data,True,typeTrain)
                val_loss += loss.item()
                val_loss_perType += loss_perType.cpu().detach().numpy()
        
        # save model
        if loss.item()<lossBest:
            lossBest = loss.item()
            torch.save({'model_state_dict': model.state_dict()},'../Model/tmp.tar')
            
        print('epoch:{}, train_loss: {:+.3f}, val_loss: {:+.3f}, \ntrain_vector: {}, \nval_vector  : {}\n'.format(epoch+epoch0,train_loss/i,val_loss/j,\
                                                            '|'.join(['%+.2f'%i for i in train_loss_perType/i]),\
                                                            '|'.join(['%+.2f'%i for i in val_loss_perType/j])))
        train_loss_list.append(train_loss_perType/i)
        val_loss_list.append(val_loss_perType/j)
        
    time_elapsed = time.time() - since
    print('Training completed in {}s'.format(time_elapsed))
    
    # load best model
    checkpoint = torch.load('../Model/tmp.tar')
    model.load_state_dict(checkpoint['model_state_dict'])
    return model,train_loss_list,val_loss_list

In [4]:
model = GNN_edgeUpdate(reuse,block,dim,layer1,layer2,factor,**data_dict[data]).to('cuda:0')

paras = trainable_parameter(model)
opt = Adam(paras,lr=lr)

In [5]:
file_name = "False_<class 'functions_refactor.MEGNet_block'>_None__data_ACSF_expand.pickle_32_128_0.4_3_3_2_20_base.tar"

In [6]:
checkpoint = torch.load('../Model/'+file_name)
model.load_state_dict(checkpoint['model_state_dict'])
opt.load_state_dict(checkpoint['optimizer_state_dict'])

In [19]:
model,train_loss_perType,val_loss_perType = train(opt,model,10,train_dl,val_dl,paras,clip)

epoch:0, train_loss: -1.343, val_loss: -1.381, 
train_vector: +0.11|-0.69|-1.17|-1.98|-2.02|-0.94|-1.84|-2.22, 
val_vector  : +0.07|-0.75|-1.23|-2.06|-1.96|-0.99|-1.94|-2.20

epoch:1, train_loss: -1.356, val_loss: -1.422, 
train_vector: +0.10|-0.68|-1.18|-1.99|-2.03|-0.95|-1.86|-2.25, 
val_vector  : -0.11|-0.88|-1.24|-2.09|-2.03|-0.96|-1.89|-2.18

epoch:2, train_loss: -1.382, val_loss: -1.394, 
train_vector: +0.05|-0.74|-1.20|-2.01|-2.05|-0.96|-1.87|-2.26, 
val_vector  : -0.08|-0.97|-1.27|-1.57|-2.05|-1.01|-1.99|-2.21

epoch:3, train_loss: -1.399, val_loss: -1.469, 
train_vector: +0.03|-0.76|-1.21|-2.03|-2.07|-0.97|-1.89|-2.28, 
val_vector  : -0.13|-0.96|-1.28|-2.12|-2.05|-1.03|-2.00|-2.19

epoch:4, train_loss: -1.415, val_loss: -1.440, 
train_vector: -0.00|-0.75|-1.23|-2.04|-2.10|-0.99|-1.91|-2.30, 
val_vector  : -0.33|-0.50|-1.26|-2.15|-2.00|-1.03|-2.01|-2.24

epoch:5, train_loss: -1.425, val_loss: -1.465, 
train_vector: -0.00|-0.76|-1.24|-2.07|-2.11|-0.99|-1.92|-2.31, 
val_vector  :

In [20]:
model,train_loss_perType,val_loss_perType = train(opt,model,10,train_dl,val_dl,paras,clip)

epoch:0, train_loss: -1.494, val_loss: -1.456, 
train_vector: -0.10|-0.84|-1.30|-2.12|-2.17|-1.05|-1.98|-2.39, 
val_vector  : +0.24|-0.93|-1.35|-2.01|-2.17|-1.08|-2.02|-2.33

epoch:1, train_loss: -1.507, val_loss: -1.568, 
train_vector: -0.12|-0.86|-1.31|-2.13|-2.19|-1.06|-1.99|-2.40, 
val_vector  : -0.22|-1.14|-1.35|-2.20|-2.14|-1.10|-2.08|-2.31

epoch:2, train_loss: -1.519, val_loss: -1.583, 
train_vector: -0.14|-0.87|-1.31|-2.16|-2.20|-1.06|-2.00|-2.41, 
val_vector  : -0.15|-1.19|-1.37|-2.29|-2.12|-1.10|-2.11|-2.35

epoch:3, train_loss: -1.532, val_loss: -1.603, 
train_vector: -0.14|-0.90|-1.33|-2.16|-2.22|-1.07|-2.02|-2.42, 
val_vector  : -0.44|-0.93|-1.39|-2.33|-2.18|-1.12|-2.08|-2.35

epoch:4, train_loss: -1.539, val_loss: -1.499, 
train_vector: -0.16|-0.88|-1.34|-2.17|-2.22|-1.08|-2.03|-2.44, 
val_vector  : -0.04|-0.47|-1.39|-2.35|-2.19|-1.11|-2.10|-2.34

epoch:5, train_loss: -1.548, val_loss: -1.523, 
train_vector: -0.16|-0.88|-1.34|-2.19|-2.24|-1.09|-2.03|-2.45, 
val_vector  :

In [21]:
model,train_loss_perType,val_loss_perType = train(opt,model,10,train_dl,val_dl,paras,clip)

epoch:0, train_loss: -1.593, val_loss: -1.631, 
train_vector: -0.21|-0.96|-1.38|-2.23|-2.27|-1.12|-2.07|-2.50, 
val_vector  : -0.32|-1.06|-1.40|-2.37|-2.25|-1.16|-2.11|-2.38

epoch:1, train_loss: -1.597, val_loss: -1.569, 
train_vector: -0.22|-0.94|-1.39|-2.21|-2.29|-1.13|-2.08|-2.51, 
val_vector  : -0.21|-0.65|-1.43|-2.38|-2.17|-1.17|-2.17|-2.38

epoch:2, train_loss: -1.608, val_loss: -1.581, 
train_vector: -0.24|-0.98|-1.39|-2.24|-2.29|-1.14|-2.09|-2.51, 
val_vector  : +0.19|-1.08|-1.42|-2.34|-2.24|-1.17|-2.19|-2.39

epoch:3, train_loss: -1.615, val_loss: -1.624, 
train_vector: -0.24|-0.98|-1.40|-2.23|-2.31|-1.14|-2.09|-2.52, 
val_vector  : -0.39|-1.15|-1.43|-2.37|-2.17|-1.17|-1.92|-2.40

epoch:4, train_loss: -1.620, val_loss: -1.612, 
train_vector: -0.24|-0.96|-1.41|-2.24|-2.31|-1.15|-2.11|-2.54, 
val_vector  : -0.45|-0.78|-1.43|-2.26|-2.24|-1.17|-2.16|-2.41

epoch:5, train_loss: -1.627, val_loss: -1.453, 
train_vector: -0.24|-0.98|-1.42|-2.25|-2.32|-1.16|-2.11|-2.54, 
val_vector  :

In [22]:
save_results(train_loss_perType,val_loss_perType,reuse,block,\
             'None',data,batch_size,dim,clip,layer1,layer2,factor,epochs)
save_model(model,opt,reuse,block,'None',data,batch_size,dim,clip,layer1,layer2,factor,epochs)

In [7]:
weight = 0.2*torch.ones(8).to('cuda:0')
weight[6] = 1

In [13]:
model,train_loss_perType,val_loss_perType = train(opt,model,10,train_dl,val_dl,paras,clip,weight=weight)

epoch:0, train_loss: -0.558, val_loss: -1.607, 
train_vector: -0.38|-1.26|-1.45|-2.41|-2.50|-1.17|-2.10|-2.68, 
val_vector  : -0.17|-0.81|-1.45|-2.40|-2.30|-1.18|-2.06|-2.48

epoch:1, train_loss: -0.541, val_loss: -1.569, 
train_vector: -0.31|-1.14|-1.41|-2.32|-2.43|-1.14|-2.06|-2.62, 
val_vector  : +0.25|-1.14|-1.43|-2.23|-2.33|-1.15|-2.05|-2.48

epoch:2, train_loss: -0.534, val_loss: -1.611, 
train_vector: -0.27|-1.09|-1.38|-2.28|-2.39|-1.12|-2.05|-2.59, 
val_vector  : -0.17|-1.06|-1.42|-2.26|-2.31|-1.14|-2.08|-2.46

epoch:3, train_loss: -0.536, val_loss: -1.586, 
train_vector: -0.27|-1.09|-1.38|-2.27|-2.38|-1.11|-2.07|-2.57, 
val_vector  : -0.28|-0.85|-1.39|-2.15|-2.32|-1.14|-2.13|-2.44

epoch:4, train_loss: -0.540, val_loss: -1.418, 
train_vector: -0.27|-1.10|-1.38|-2.27|-2.38|-1.11|-2.10|-2.58, 
val_vector  : +0.66|-0.36|-1.40|-2.27|-2.28|-1.14|-2.12|-2.44

epoch:5, train_loss: -0.537, val_loss: -1.653, 
train_vector: -0.24|-0.94|-1.38|-2.25|-2.38|-1.11|-2.12|-2.57, 
val_vector  :

In [14]:
save_results(train_loss_perType,val_loss_perType,reuse,block,\
             'None',data,batch_size,dim,clip,layer1,layer2,factor,epochs)
save_model(model,opt,reuse,block,'None',data,batch_size,dim,clip,layer1,layer2,factor,epochs)

### make submissions

In [14]:
make_submission(reuse,block,head,data,batch_size,dim,clip,layer1,layer2,factor,epochs)

From start

In [3]:
head = cat3Head_type

In [4]:
make_submission(reuse,block,head,data,batch_size,dim,clip,layer1,layer2,factor,epochs,'_start')

2Stage

In [3]:
head = cat3Head_type

In [4]:
make_submission(reuse,block,head,data,batch_size,dim,clip,layer1,layer2,factor,epochs,'_2stage')